# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [50]:
!cat publications.tsv







2022-07-31	On the robustness of the metric dimension of grid graphs to adding a single edge	Discrete Applied Mathematics	"We prove that if we add an extra edge to a (large enough) d-dimensional grid graph, then the resulting graph will have metric dimension between d and 2d, and we almost completely settle the case for d=2."	"Satvik Mashkaria, Gergely �dor, Patrick Thiran"	robustness	https://doi.org/10.1016/j.dam.2022.02.014	png

## Import pandas

We are using the very handy pandas library for dataframes.

In [51]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [52]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0,encoding="mac-roman")
publications


,pub_date,title,venue,excerpt,authors,url_slug,paper_url,imageformat
0,2021-10-14,Switchover phenomenon induced by epidemic seed...,Proceedings of the National Academy of Sciences,We observe empirically and prove theoretically...,"Gergely Ódor, Domonkos Czifra, Júlia Komjáthy,...",switchover,https://doi.org/10.1073/pnas.2112607118,png
1,2022-04-08,The power of adaptivity in source identificati...,Theoretical Computer Science,We provide the first theoretical results on th...,"Victor Lecomte, Gergely Ódor, and Patrick Thiran",power,https://doi.org/10.1016/j.tcs.2022.02.008,png
2,2021-11-21,Sequential metric dimension for random graphs,Journal of Applied Probability,We prove that the sequential version of the me...,Gergely Ódor and Patrick Thiran,smd,https://doi.org/10.1017/jpr.2021.16,png
3,2021-06-01,Metric dimension of critical Galton-Watson tre...,European Journal of Combinatorics,"Building on the literature of fringe trees, we...","Júlia Komjáthy, Gergely Ódor",fringe,https://doi.org/10.1016/j.ejc.2021.103317,png
4,2021-12-29,Source Detection via Contact Tracing in the Pr...,arxiv,We define and prove rigorous results about a n...,"Gergely Ódor, Jana Vuckovic, Miguel-Angel Sanc...",sictf,https://arxiv.org/abs/2112.14530,png
5,2022-07-31,On the robustness of the metric dimension of g...,Discrete Applied Mathematics,We prove that if we add an extra edge to a (la...,"Satvik Mashkaria, Gergely Ódor, Patrick Thiran",robustness,https://doi.org/10.1016/j.dam.2022.02.014,png


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [53]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [54]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\nlink: """ + item.paper_url
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nauthors: '" + html_escape(item.authors) + "'"
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    md += "\nimage: '" + html_escape(item.url_slug) +"." + html_escape(item.imageformat) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
            
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [48]:
!ls ../_publications/

2021-06-01-fringe.md     2021-11-21-smd.md        2022-07-31-robustness.md
2021-10-14-swichover.md  2021-12-29-sictf.md
2021-10-14-switchover.md 2022-04-08-power.md
